In [1]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
    border-style: groove;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: auto;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h1>
<center>
Final Project. Callin out Twitter Haters
</center>
</h1>
<div class=h1_cell>
<p>
Using the bag of tweets from week 3 as our training data, I will scrape Twitter's trending hashtags/stories and find haters. From the list of haters that are found from trending, I'll post to a status here: https://twitter.com/HDetection/with_replies. You'll need to install the following packages that are listed in the code below
</div>

In [1]:
# pip install twitter
# pip install msgpack
# pip install argparse

import pandas as pd
import os
import re
import twitter
from twitter import *
import json
home_path =  os.path.expanduser('~')
file_path = '/Documents/CIS/UpperDiv/CIS_399/final/'
file_name = 'tweet_table.csv'

tweet_table = pd.read_csv(home_path + file_path + file_name)

In [2]:
token = "Top Secret"
token_secret = "Hush Hush information"
consumer_key = "Shhhhhhhh"
consumer_secret = "Secret McSecretFace"
t = Twitter(
    auth=OAuth(token, token_secret, consumer_key, consumer_secret))

In [3]:
tweet_table.head()

,id,label,tweet,length,bang_count,hash_count,#trump,#politics,#allahsoil,#libtard,#liberal
0,1,0,@user when a father is dysfunctional and is so...,101,0,1,0,0,0,0,0
1,2,0,@user @user thanks for #lyft credit i can't us...,122,0,3,0,0,0,0,0
2,3,0,bihday your majesty,19,0,0,0,0,0,0,0
3,4,0,#model i love u take with u all the time in ...,116,3,1,0,0,0,0,0
4,5,0,factsguide: society now #motivation,38,0,1,0,0,0,0,0


<div class=h1_cell>
<p>
Set up from earlier in the term. Naive Bayes, count all hashes, useful counts, etc.
<p>
</div>

In [4]:
def count_all_hashes(table):
    hashes_Dict = {}
    hashTags = []
    hash_pat = r"#\w+"
    for i in range(len(table)):
        hashTags = re.findall(hash_pat, table.iloc[i]['tweet'].lower())
        label = table.iloc[i]['label']
        for j in range(len(hashTags)):
            if hashTags[j] not in hashes_Dict:
                hashes_Dict[hashTags[j]] = [0, 0]
            hashes_Dict[hashTags[j]][label] += 1
    return hashes_Dict    

In [7]:
useful_counts = {}
hateCount = 0
noHateCount = 0
for i in range(len(tweet_table)):
    if (tweet_table.iloc[i]['label'] == 0):
        noHateCount += 1
    else:
        hateCount += 1

total_tweets = len(tweet_table)

useful_counts['class_count'] = (noHateCount, hateCount)

noHateProb = noHateCount / float(total_tweets)
hateProb = hateCount / float(total_tweets)

useful_counts['class_prob'] = (noHateProb, hateProb)
useful_counts['tweet_count'] = total_tweets

useful_counts

{'class_count': (29720, 2242),
 'class_prob': (0.9298542018647143, 0.07014579813528565),
 'tweet_count': 31962}

In [8]:
hashTags = []
hash_pat = r"#\w+"
nakedNoHate = 0
nakedHate = 0
for i in range(len(tweet_table)):
    hashTags = re.findall(hash_pat, tweet_table.iloc[i]['tweet'].lower())
    label = tweet_table.iloc[i]['label']
    if(label == 0 and len(hashTags) == 0):
        nakedNoHate += 1
    elif(label == 1 and len(hashTags) == 0):
        nakedHate += 1

useful_counts['naked_count'] = (nakedNoHate, nakedHate)
useful_counts

{'class_count': (29720, 2242),
 'class_prob': (0.9298542018647143, 0.07014579813528565),
 'naked_count': (7933, 608),
 'tweet_count': 31962}

<div class=h1_cell>
<p>
Problem: everytime we load this notebook, we lost our data from our previous session, all the hashtags/useful_counts get reset </p>
<br>
<p>Solution: read in files that I wrote out that saves this data</p>
</div>

In [3]:
# list of saved hashtags, this includes those from the table and new ones found from twitter
# saved useful_counts so that way we don't start fresh everytime we load up this tool
all_hashes = json.load(open("bag_of_hashes.txt"))
useful_counts = json.load(open("useful_counts.txt"))

<div class=h1_cell>
    <p>
    Bring in Naive Bayes and create a function called update_useful_counts to account for the new tweets being found
    </p>
</div>

In [4]:
def update_useful_counts(count_dict, hate_value):
        newClassCount = [count_dict['class_count'][0], count_dict['class_count'][1]]
        newClassCount[hate_value] += 1
        count_dict['class_count'] = tuple(newClassCount)
        newClassProb = [count_dict['class_count'][0] / float(count_dict['tweet_count']), count_dict['class_count'][1] / float(count_dict['tweet_count'])]
        count_dict['class_prob'] = tuple(newClassProb)

hash_pat = r"#\w+"
hash_patc = re.compile(hash_pat)
def naive_bayes(text, count_dictionary, patc, bag_of_words, class_size):
    valuesInText = re.findall(patc, text)
    countValues = [1] * class_size
    for i in range(len(valuesInText)):
        if valuesInText[i] in bag_of_words:
            for j in range(class_size):
                countValues[j] *= bag_of_words[valuesInText[i]][j] / float(count_dictionary['class_count'][j])
    
    if (len(valuesInText) == 0):
        for i in range(class_size):
            countValues[i] *= count_dictionary['naked_count'][i] / float(count_dictionary['class_count'][i])
    
    finalValues = [0] * class_size
    for i in range(class_size):
        finalValues[i] = countValues[i] * count_dictionary['class_prob'][i]
    
    # i can add this in since i will no longer be using the training data and will be reading in real data. If a tweet has no hashtags
    # then after computing its value as either hate or not, we should increment the naked_count
    hateVal = 0 if finalValues[0] > finalValues[1] else 1
    if len(valuesInText) == 0:
        newNakedCount = [count_dictionary['naked_count'][0], count_dictionary['naked_count'][1]]
        newNakedCount[hateVal] += 1
        count_dictionary['naked_count'] = tuple(newNakedCount)
    for value in valuesInText:
        if value not in bag_of_words:
            bag_of_words[value] = [0,0]
        bag_of_words[value][hateVal] += 1
    return tuple(finalValues)


In [11]:
#list of trends for the united states
results = t.trends.place(_id=23424977)
for location in results:
    for trend in location['trends']:
        print trend['name']

Anthony Bourdain
#NationalBestFriendsDay
#WorldOceansDay
Charles Krauthammer
Kids See Ghosts
#FridayFeeling
#HalloweenMovie
Gronk
Muhammad Ali
#FlashbackFriday
Justice Sotomayor
Michael Busch
ON SALE NOW
Yoel Romero
Caden O'Brien
Tar Heels
Delpo
Soviet-style
#AnnoyATVShow
#USCM2018
#NewMusicFriday
#StanleyCupChampions
#MyStory
#BreyersBFF
#LMS18
#EndTheStigma
#GhostbustersDay
#ImpactThatMatters
#DXC600
#TravelSkills
#FurnitureASongOrBand
#MIT2018
#VHSL
#OMCchat
#JakksIncredibles2


<div class=h1_cell>
    <p>
        Loop through trends in the United States, find tweets that are in English (I think I may have labeled someone from a scandinavian country a hater despite not knowing what the tweet said, so we now search solely for english tweets), go through the tweets and put them through naive_bayes, if a hate tweet is found then we print the tweet and append to our haterList and lastly we update our useful_counts. Possible that no haters will be found from one go.
    <br><br>EDIT: Made this into a function and have it loop until we hit the rate limit. :)
    </p>
    </div>

In [18]:
# woeid for the united states
# very well could find no haters since the # of tweets returned only gives about ~150 total tweets across all tweets
haterList = []
def scrapeTwitter():
    trend_result = t.trends.place(_id=23424977)
    badTweetList = []
    while True:
        for location in trend_result:
            for trending in location['trends']:
                try:
                    tweets = t.search.tweets(q=trending['name'], lang='en')
                except:
                    print "rate limit reached"
                    return badTweetList
                for statuses in tweets['statuses']:
                    value = naive_bayes(statuses['text'], useful_counts, hash_pat, all_hashes, 2)
                    useful_counts['tweet_count'] += 1
                    if value[1] > value[0]:
                        badTweetList.append(statuses['text'])
                        badTweetList = list(set(badTweetList))
                        haterList.append(statuses['user']['screen_name'])
                    hateVal = 0 if value[0] > value[1] else 1
                    if statuses['text'] not in tweetList:
                        update_useful_counts(useful_counts, hateVal) 
    return badTweetList

In [19]:
tweetList = scrapeTwitter()
haterList = list(set(haterList))
print tweetList
print haterList

rate limit reached
[]
[]


In [15]:
def veto(hatersToRemove):
    for hater in hatersToRemove:
        haterList.remove(hater)

def reverseUseful():
    reversedCount = [useful_counts['class_count'][0]+1, useful_counts['class_count'][1]-1]
    useful_counts['class_count'] = tuple(reversedCount)
    correctedProb = [useful_counts['class_count'][0] / float(useful_counts['tweet_count']), useful_counts['class_count'][1] / float(useful_counts['tweet_count'])]
    useful_counts['class_prob'] = tuple(correctedProb)

In [20]:
# ONLY NEEDED IF WE DETERMINE A USER IS MIS-LABELED A HATER
notHaters = []
notHaters.append(None)
veto(notHaters)
reverseUseful()

ValueError: list.remove(x): x not in list

<div class=h1_cell>
    <p>
        Loop through our haters, put a status on twitter, then clear out our haterList so we can find new ones. All evidence of haters can be found in the twitter link posted at the top :)
    </p>
    </div>

In [16]:
# post message to twitter saying that a hater has been found
for hater in haterList:
    message = "@" + hater + " has been found to be a hater"
    try:
        t.statuses.update(status=message)
    except:
        print hater + ' status already exists'

# clear out the haterList for new haters to be found.
haterList = []
tweetList = []

In [21]:
#updated useful_counts
useful_counts

{u'class_count': (43524, 2307),
 u'class_prob': (0.9496007330802461, 0.05033381332635162),
 u'naked_count': (19712, 608),
 u'tweet_count': 45834}

<div class=h1_cell>
<p>
    write our new list of hashtags and our most current useful_counts out to their own files so we don't lose important data
    </p>
    </div>

In [22]:
with open('bag_of_hashes.txt', 'w') as file:
    file.write(json.dumps(all_hashes))
    
with open('useful_counts.txt', 'w') as file:
    file.write(json.dumps(useful_counts))